In [ ]:
# Necessary imports
import pandas as pd
import numpy as np
import os.path as path
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#load the data into dataframe
my_path = path.abspath(path.join("__file__","../.."))
rel_path = 'Data\phq_all_final.csv'
csv_path = path.join(my_path,rel_path)
df = pd.read_csv(csv_path)

In [ ]:
#View the sample records
df.head()

In [ ]:
#Sort the records according to Patient_ID
df1 = df.sort_values(['patient_id','date'],ignore_index=True)
df1

In [ ]:
#Convert Dates from String to Datetime
df1['date'] = pd.to_datetime(df1['date'])
df1['patient_date_created'] = pd.to_datetime(df1['patient_date_created'])
df1.head()

In [ ]:
# Extract only dates
df1['date'] = df1['date'].apply(lambda x:x.date().strftime('%m-%d-%Y'))
df1['patient_date_created'] = df1['patient_date_created'].apply(lambda x:x.date().strftime('%m-%d-%Y'))

In [ ]:
# Drop duplicates which are multiple assessments in a day and retain the latest score
df1 = df1.drop_duplicates(subset=['patient_id','date'],keep='last').reset_index(drop=True)
df1

In [ ]:
# Get the Patient_ID from user and extract records related to that patient only

# Sample Patient IDs to see different cases = [2161, 6917, 7561, 48, 9]

p_id = input("Enter the patient ID ") # Run the cell, enter the patient ID and press enter
data = df1.query('patient_id==@p_id')
print('p_id')
data

In [ ]:
class tracker:
    
    def __init__(self,data):
        self.data = data
        
    
    # status metric logic that shows different statuses related to the score
    def status(self):
        
        l = list(self.data['score'])
        status = ''           
        
        #consider only last 4 scores.
        #Check if the patient has atleast 4 scores
        if len(l) >=4:
            
            # check if difference between 4th last score and 3rd last score is a zone apart(lower Serverity Zone)
            # check if last 3 scores lie in the same severity zone
            
            if l[-4] - l[-3] >=5:
                if 0 <= abs(l[-3] -l[-2]) <5 and 0 <= abs(l[-3]-l[-1])<5:
                    status = 'Improved'   
                else:
                    status = "Inconsistent"
                    
            # check if difference between 3rd last score and 4th last score is a zone apart(Higher Severity Zone)  
            
            elif l[-3] - l[-4] >=5:
                if 0 <= abs(l[-3] -l[-2]) <5 and 0 <= abs(l[-3]-l[-1]) <5:
                    status = "Increased Severity"
                else:
                    status = "Inconsistent"
                    
            # check if difference between 4th last and 3rd last score lie in the same severity zone  
            # check if last 3 scores lie in the same severity zone
            # check if last 2 scores lie in the same severity zone but not 3rd last
            
            elif 0 <= abs(l[-4] - l[-3]) <5:
                 
                if 0<= abs(l[-3] -l[-2]) <5 and 0 <=abs(l[-3]-l[-1])<5:
                    status = "No Change "
                elif l[-3] - l[-2] >=5 and l[-3] - l[-1] >=5:
                    status = "Improved"
                elif l[-2] - l[-3] >=5 and l[-1] - l[-3] >=5:
                    status = "Worsened"
                else:
                    status = "Inconsistent"
                    
        # Patient has less than 4 scores           
        else:
            status = "Need more sessions to track progress"
        
        
        return status
    
    
    def visualize(self):
        
        # Line plot using date and score
        plt.rcParams["figure.figsize"] = (15,7)
        plt.plot(self.data['date'],self.data['score'],marker='o',linewidth=3)
        plt.yticks(np.arange(0,26,5),label='Score')  # custom y axis scale
        plt.xticks(rotation=45)
        plt.xlabel('Date')
        plt.ylabel('Score')
        
        # Grid lines(in the interval of 5) for Y axis to represent the severity zones 
        plt.grid(which='major', axis = 'y', linestyle = '-.', linewidth =0.75,color='black')
        plt.title('Patient_ID: {} created on {}. Status - {}'.format(p_id,str(self.data['patient_date_created'].unique()[0]),self.status()),fontsize=20)
        
        x0,x1 = plt.gca().get_xlim()
        plt.axhline(y=10,linewidth=3, color='orange',linestyle='solid') # Threshold line
        plt.text(x1,10, " Further Clinical Evaluation Threhold", ha='left', va='center')
        plt.savefig('Patient_ID_{}_Progress_Chart.png'.format(p_id),bbox_inches='tight')

In [ ]:
def main():
    t1 = tracker(data) # Class object
    t1.visualize()
    

if __name__ == "__main__":
    main()

In [ ]:
"""I am defining "improvement/progress" as 
case 1: a change in severity zone(lower) from 4th last score to 3rd last score and 
        the score stays in the same severity zone for the next 3 consecutive sessions
        
case 2: 4th last score and 3rd last score lie in the same severity zone but
        last 2 scores fall in decereased severity zone"""